<a href="https://colab.research.google.com/github/Only-Mike/ADHD/blob/main/ADHD_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Import necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [2]:
!git clone 'https://github.com/Only-Mike/ADHD.git'

Cloning into 'ADHD'...
remote: Enumerating objects: 856, done.
remote: Counting objects: 100% (373/373), done.
remote: Compressing objects: 100% (184/184), done.
remote: Total 856 (delta 225), reused 303 (delta 189), pack-reused 483
Receiving objects: 100% (856/856), 17.09 MiB | 10.87 MiB/s, done.
Resolving deltas: 100% (471/471), done.


In [3]:
df = pd.read_csv('/content/ADHD/datasets/KKI_phenotypic.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83 entries, 0 to 82
Data columns (total 23 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ScanDir ID       83 non-null     int64  
 1   Site             83 non-null     int64  
 2   Gender           83 non-null     int64  
 3   Age              83 non-null     float64
 4   Handedness       83 non-null     int64  
 5   DX               83 non-null     int64  
 6   Secondary Dx     15 non-null     object 
 7   ADHD Measure     83 non-null     int64  
 8   ADHD Index       83 non-null     int64  
 9   Inattentive      83 non-null     int64  
 10  Hyper/Impulsive  83 non-null     int64  
 11  IQ Measure       83 non-null     int64  
 12  Verbal IQ        83 non-null     int64  
 13  Performance IQ   83 non-null     int64  
 14  Full2 IQ         0 non-null      float64
 15  Full4 IQ         83 non-null     int64  
 16  Med Status       83 non-null     int64  
 17  QC_Rest_1        8

In [5]:
#dropping unrelevant columns
df = df.drop(columns = ['Site', 'ADHD Measure', 'IQ Measure', 'Full2 IQ', 'QC_Rest_1', 'QC_Rest_2', 'QC_Rest_3', 'QC_Rest_4', 'QC_Anatomical_1', 'QC_Anatomical_2', 'DX'])

In [6]:
df.describe() #We have some false values in the following columns: ADHD Index, Inattentive and Hyper/Impulsive

,ScanDir ID,Gender,Age,Handedness,ADHD Index,Inattentive,Hyper/Impulsive,Verbal IQ,Performance IQ,Full4 IQ,Med Status
count,8.300000e+01,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000
mean,3.449295e+06,0.554217,10.243253,0.927711,14.963855,15.265060,15.734940,113.060241,108.686747,110.012048,1.180723
std,2.098556e+06,0.500073,1.346601,0.303762,198.036178,198.098958,198.141738,14.693909,11.995861,11.935287,0.387128
min,1.018959e+06,0.000000,8.020000,0.000000,-999.000000,-999.000000,-999.000000,81.000000,79.000000,85.000000,1.000000
25%,2.005148e+06,0.000000,9.100000,1.000000,42.000000,42.000000,43.000000,104.000000,102.000000,101.500000,1.000000
50%,2.768273e+06,1.000000,10.120000,1.000000,46.000000,48.000000,48.000000,112.000000,108.000000,111.000000,1.000000
75%,3.915209e+06,1.000000,11.095000,1.000000,61.000000,60.000000,59.500000,121.500000,119.000000,119.000000,1.000000
max,9.922944e+06,1.000000,12.990000,2.000000,90.000000,90.000000,90.000000,146.000000,137.000000,134.000000,2.000000


In [7]:
#Round age for fewer unique values and making into integer
df['Age'] = df['Age'].round(decimals = 0)
df['Age'] = df['Age'].astype(int)

In [8]:
#Removes rows with -999 in the following columns
df = df[df['Inattentive'] != -999]
df = df[df['Hyper/Impulsive'] != -999]
df = df[df['ADHD Index'] != -999]

In [9]:
#But as we can see with the min function, it removed the false values of -999
df.min()

<ipython-input-9-86b8d0be8a1f>:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.min()


ScanDir ID         1018959
Gender                   0
Age                      8
Handedness               0
ADHD Index              40
Inattentive             40
Hyper/Impulsive         41
Verbal IQ               81
Performance IQ          79
Full4 IQ                85
Med Status               1
dtype: int64

In [10]:
df['Secondary Dx '].unique()

array([nan, 'Simple phobia', 'Simple Phobia', 'simple phobias', 'ODD',
       'Simple Phobia ', 'ODD; Phobia', 'Specific phobia', 'Phobia',
       'social and simple phobia '], dtype=object)

In [11]:
#Making none secondary dx into 0 and any secondary dx into 1
df['Secondary Dx '].replace(('Simple phobia', 'Simple Phobia', 'simple phobias', 'ODD', 'Simple Phobia ', 'ODD; Phobia', 'Specific phobia', 'Phobia', 'social and simple phobia '), (1, 1, 1, 1, 1, 1, 1, 1, 1), inplace=True)
df['Secondary Dx '] = df['Secondary Dx '].fillna(0).astype(int)

In [12]:
#Checking that it worked as intended
df['Secondary Dx '].unique()

array([0, 1])

In [13]:
pip install sdv -q

     |████████████████████████████████| 102 kB 3.3 MB/s 
     |████████████████████████████████| 47 kB 3.9 MB/s 
     |████████████████████████████████| 53 kB 1.4 MB/s 
     |████████████████████████████████| 61 kB 336 kB/s 
     |████████████████████████████████| 139 kB 41.6 MB/s 
     |████████████████████████████████| 1.6 MB 70.1 MB/s 
     |████████████████████████████████| 9.4 MB 60.1 MB/s 
     |████████████████████████████████| 965 kB 57.7 MB/s 
     |████████████████████████████████| 295 kB 77.1 MB/s 
     |████████████████████████████████| 662 kB 65.4 MB/s 
     |████████████████████████████████| 280 kB 59.1 MB/s 
     |████████████████████████████████| 15.3 MB 48.7 MB/s 


In [14]:
from sdv.tabular import GaussianCopula
model = GaussianCopula()
model.fit(df)

In [15]:
#Creating the synthetic data
synthetic_data = model.sample(2000)
synthetic_data.head()

,ScanDir ID,Gender,Age,Handedness,Secondary Dx,ADHD Index,Inattentive,Hyper/Impulsive,Verbal IQ,Performance IQ,Full4 IQ,Med Status
0,3617659,1,13,1,1,63,46,73,116,90,97,1
1,3889981,0,8,0,1,68,73,82,94,85,105,2
2,1671890,1,13,1,0,58,51,55,96,119,112,1
3,3405687,0,12,1,0,75,67,68,122,110,105,1
4,4203621,1,9,1,0,64,65,76,127,101,112,1


In [16]:
#Checking the number of unique values in each column to see if something seems odd
synthetic_data.nunique()

ScanDir ID         2000
Gender                2
Age                   6
Handedness            3
Secondary Dx          2
ADHD Index           51
Inattentive          51
Hyper/Impulsive      50
Verbal IQ            66
Performance IQ       59
Full4 IQ             50
Med Status            2
dtype: int64

In [17]:
#Appending the synthetic data to the original df
df = df.append([synthetic_data])

In [18]:
#There is now 2080 rows each with a unique id
df.nunique()

ScanDir ID         2080
Gender                2
Age                   6
Handedness            3
Secondary Dx          2
ADHD Index           51
Inattentive          51
Hyper/Impulsive      50
Verbal IQ            66
Performance IQ       59
Full4 IQ             50
Med Status            2
dtype: int64

# Scaling the data

In [19]:
scaler = MinMaxScaler()

In [20]:
scaler.fit(df)

MinMaxScaler()

In [21]:
scaler.transform(df)

array([[0.15185032, 0.        , 0.6       , ..., 0.68965517, 0.75510204,
        0.        ],
       [0.11311272, 0.        , 1.        , ..., 0.5       , 0.42857143,
        0.        ],
       [0.27129639, 0.        , 0.        , ..., 0.32758621, 0.08163265,
        0.        ],
       ...,
       [0.31069167, 0.        , 0.2       , ..., 0.5       , 0.57142857,
        0.        ],
       [0.09103216, 0.        , 0.4       , ..., 0.84482759, 0.95918367,
        0.        ],
       [0.21389591, 1.        , 0.4       , ..., 0.4137931 , 0.40816327,
        1.        ]])

# ADHD Index vs Inattentive vs Hyper/Impulsive

In [22]:
corr_matrix = df.corr()
corr_matrix['ADHD Index'].sort_values(ascending = False)

ADHD Index         1.000000
Inattentive        0.843844
Hyper/Impulsive    0.721476
Secondary Dx       0.190946
Med Status         0.093772
Age                0.059420
Handedness        -0.009890
ScanDir ID        -0.012829
Verbal IQ         -0.016426
Performance IQ    -0.087218
Full4 IQ          -0.156579
Gender            -0.170672
Name: ADHD Index, dtype: float64

In [23]:
df.corr()

,ScanDir ID,Gender,Age,Handedness,Secondary Dx,ADHD Index,Inattentive,Hyper/Impulsive,Verbal IQ,Performance IQ,Full4 IQ,Med Status
ScanDir ID,1.000000,0.141253,-0.117777,0.051813,0.072104,-0.012829,-0.030737,-0.105137,-0.026519,-0.039193,-0.022107,-0.052680
Gender,0.141253,1.000000,0.045809,-0.096864,0.000989,-0.170672,-0.234151,-0.117562,0.041961,0.037957,-0.098947,-0.090385
Age,-0.117777,0.045809,1.000000,0.046848,0.031740,0.059420,-0.019902,0.122049,-0.047769,-0.121135,-0.155562,-0.106719
Handedness,0.051813,-0.096864,0.046848,1.000000,0.003849,-0.009890,-0.005332,0.036349,-0.014386,-0.038256,0.007523,0.071486
Secondary Dx,0.072104,0.000989,0.031740,0.003849,1.000000,0.190946,0.197257,0.196442,0.114127,-0.015057,0.113532,0.000819
ADHD Index,-0.012829,-0.170672,0.059420,-0.009890,0.190946,1.000000,0.843844,0.721476,-0.016426,-0.087218,-0.156579,0.093772
Inattentive,-0.030737,-0.234151,-0.019902,-0.005332,0.197257,0.843844,1.000000,0.702901,-0.008072,-0.056772,-0.023203,0.092007
Hyper/Impulsive,-0.105137,-0.117562,0.122049,0.036349,0.196442,0.721476,0.702901,1.000000,-0.042184,-0.123026,-0.107650,0.163062
Verbal IQ,-0.026519,0.041961,-0.047769,-0.014386,0.114127,-0.016426,-0.008072,-0.042184,1.000000,0.156872,0.559913,-0.159439
Performance IQ,-0.039193,0.037957,-0.121135,-0.038256,-0.015057,-0.087218,-0.056772,-0.123026,0.156872,1.000000,0.489066,-0.012035


In [24]:
from pandas.plotting import scatter_matrix

attributes = ["Hyper/Impulsive", "ADHD Index", "Inattentive"]
scatter_matrix(df[attributes], figsize = (16,12))
plt.show()

ImportError: ignored

<Figure size 1152x864 with 9 Axes>

#Linear Regression

In [25]:
# Assign X and y for training
X = df[["Inattentive", "Hyper/Impulsive", "Secondary Dx ", "Gender", "Age", "Handedness"]].values
y = df["ADHD Index"]

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 43)

In [27]:
model = LinearRegression()

In [28]:
model.fit(X_train, y_train)

LinearRegression()

In [29]:
X_new = [[75, 90, 1, 1, 20, 2]]
print(model.predict(X_new))


[80.1984077]


In [30]:
print(model.score(X_test, y_test))

0.73236098509688


# Hard coding features in the app

In [31]:
if model.predict(X_new) >= 70:
    print('It seems from your answers, you might have difficulties with your diagnosis. We would recommend signing up for all our listed features...')
elif model.predict(X_new) >= 40 or model.predict(X_new) <= 69:
    print('With your ADHD Index we recommend using our psychoeducation and coaching feature')
else:
    print('With your ADHD Index we would recommend using our psychoeducation feature...')

It seems from your answers, you might have difficulties with your diagnosis. We would recommend signing up for all our listed features...


#K-Nearest Neighbors


In [32]:
from sklearn.neighbors import KNeighborsRegressor

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 21)

In [33]:
knn_model = KNeighborsRegressor(n_neighbors=3)

In [34]:
knn_model.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=3)

In [35]:
knn_X_new = [[75, 90, 1, 1, 20, 2]]
print(knn_model.predict(knn_X_new))


[75.66666667]


In [36]:
print(knn_model.score(X_test, y_test))

0.6655337457377275


In [37]:
if model.predict(X_new) >= 60:
    print('You most likely have ADHD or ADD')
else: 
    print('You dont have ADHD')


You most likely have ADHD or ADD


# Random Forest Regressor

In [38]:
from sklearn.ensemble import RandomForestRegressor

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 21)

In [39]:
rfg_model = RandomForestRegressor()

In [40]:
rfg_model.fit(X_train, y_train)

RandomForestRegressor()

In [41]:
rfg_X_new = [[75, 90, 1, 1, 20, 2]]
print(rfg_model.predict(rfg_X_new))

[77.14]


In [42]:
print(rfg_model.score(X_test, y_test))

0.682230162339631


In [43]:
if model.predict(X_new) >= 60:
    print('You most likely have ADHD or ADD')
else: 
    print('You dont have ADHD')

You most likely have ADHD or ADD
